<a href="https://colab.research.google.com/github/maxotuteye/MLandAI/blob/main/WorkingWIthGANs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from keras.layers import Input

In [ ]:
from keras.layers import LeakyReLU as LK

In [ ]:
from keras.models import Model, Sequential

In [ ]:
from keras.layers.core import Reshape, Dense, Dropout, Flatten

In [ ]:
from keras.layers.normalization import BatchNormalization

In [ ]:
from keras.datasets import mnist

In [ ]:
from keras.optimizers import Adam

In [ ]:
from keras import backend as K

In [ ]:
from keras import initializers

In [ ]:
K.set_image_data_format('channels_first')

In [ ]:
np.random.seed(1000)

In [ ]:
randomDim = 100

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
X_train = (X_train.astype(np.float32) - 127.5)/127.5

X_train = X_train.reshape(60000, 784)

In [ ]:
adam = Adam(lr=0.0002, beta_1=0.5)

In [ ]:
generator = Sequential()

In [ ]:
generator.add(Dense(256, input_dim=randomDim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))

In [ ]:
generator.add(LK(0.2))

In [ ]:
generator.add(Dense(512))

In [ ]:
generator.add(LK(0.2))

In [ ]:
generator.add(Dense(1024))

In [ ]:
generator.add(LK(0.2))

In [ ]:
generator.add(Dense(784, activation='tanh'))

In [ ]:
generator.compile(loss='binary_crossentropy',
                  optimizer=adam)

In [ ]:
discriminator = Sequential()

In [ ]:
discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.2)))

In [ ]:
discriminator.add(LK(0.2))

In [ ]:
discriminator.add(Dropout(0.3))

In [ ]:
discriminator.add(Dense(512))

In [ ]:
discriminator.add(LK(0.2))

In [ ]:
discriminator.add(Dropout(0.3))

In [ ]:
discriminator.add(Dense(256))

In [ ]:
discriminator.add(LK(0.2))

In [ ]:
discriminator.add(Dropout(0.3))

In [ ]:
discriminator.add(Dense(1, activation='sigmoid'))

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
discriminator.trainable = False

In [ ]:
ganInput = Input(shape=(randomDim,))

In [ ]:
x = generator(ganInput)

In [ ]:
ganOutput = discriminator(x)

In [ ]:
gan = Model(inputs=ganInput, outputs=ganOutput)

In [ ]:
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
dLosses = []
gLosses = []

In [ ]:
def plotLoss(epoch):
  plt.figure(figsize=(10,8))
  plt.plot(dLosses, label='Discriminative loss')
  plt.plot(gLosses, label='Generative loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  #plt.savefig('images/gan_loss_epoch_%d.png' %epoch)

In [ ]:
def plotGeneratedImages(epoch, examples=100, dim=(10,10), figsize=(10,10)):
  noise = np.random.normal(0, 1, size=[examples,
  randomDim])
  generatedImages = generator.predict(noise)
  generatedImages = generatedImages.reshape(examples, 28, 28)
  plt.figure(figsize=figsize)

  for i in range(generatedImages.shape[0]):
    plt.subplot(dim[0], dim[1], i+1)
    plt.imshow(generatedImages[i],
    interpolation='nearest', cmap='gray_r')
    plt.axis('off')
    plt.tight_layout()
    #plt.savefig('images/gan_generated_image_epoch_%d.png' % epoch)

In [ ]:
def saveModels(epoch):
  generator.save('models/gan_generator_epoch_%d.h5' % epoch)
  discriminator.save('models/gan_discriminator_epoch_%d.h5' % epoch)

In [ ]:
def train(epochs=1, batchSize=128):
  batchCount = X_train.shape[0] / batchSize
  print ('Epochs:', epochs)
  print ('Batch size:', batchSize)
  print ('Batches per epoch:', batchCount)
  for e in range(1, epochs+1):
    print ('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(int(batchCount))):
      noise = np.random.normal(0, 1,size=[batchSize, randomDim])
      imageBatch = X_train[np.random.randint(0, X_train.shape[0],size=batchSize)]
      generatedImages = generator.predict(noise)
      np.shape(generatedImages)
      X = np.concatenate([imageBatch, generatedImages])
      yDis = np.zeros(2*batchSize)
      yDis[:batchSize] = 0.9
      discriminator.trainable = True
      dloss = discriminator.train_on_batch(X, yDis)
      noise = np.random.normal(0, 1, size=[batchSize, randomDim])
      yGen = np.ones(batchSize)
      discriminator.trainable = False
      gloss = gan.train_on_batch(noise, yGen)
    dLosses.append(dloss)
    gLosses.append(gloss)
    if e == 1 or e % 20 == 0:
       plotGeneratedImages(e)
       #saveModels(e)
     
  plotLoss(e)
train(200,128)


  0%|          | 1/468 [00:00<01:18,  5.96it/s]

Epochs: 200
Batch size: 128
Batches per epoch: 468.75
--------------- Epoch 1 ---------------



100%|██████████| 468/468 [01:22<00:00,  5.70it/s]

  0%|          | 1/468 [00:00<01:20,  5.78it/s]

--------------- Epoch 2 ---------------



100%|██████████| 468/468 [01:23<00:00,  5.60it/s]

  0%|          | 1/468 [00:00<01:22,  5.63it/s]

...


100%|██████████| 468/468 [01:20<00:00,  5.78it/s]

  0%|          | 1/468 [00:00<01:16,  6.14it/s]

--------------- Epoch 40 ---------------



100%|██████████| 468/468 [01:22<00:00,  5.70it/s]

  0%|          | 1/468 [00:00<01:20,  5.81it/s]

--------------- Epoch 41 ---------------



 13%|█▎        | 63/468 [00:11<01:11,  5.67it/s]